### Capstone project - Applied Data Science Capstone by IBM/Coursera
___
# The best inner London areas to launch new coffee shops

### Import required libraries

In [1]:
import pandas as pd
!pip install openpyxl

import numpy as np # library for vectorized computation

# Create a map of London for plotting
!conda install -c conda-forge folium --y
import folium # map rendering library

# convert an address into latitude and longitude values
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

from sklearn.cluster import KMeans # import k-means from clustering stage

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import requests # 'requests' offers the most friendly API for opening files, including JSON support
import json # library to handle JSON files
from pandas import json_normalize # tranform JSON file into a pandas dataframe

print('Libraries imported.')

     |████████████████████████████████| 245kB 17.8MB/s eta 0:00:01
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    ca-certificates-2020.12.5  |       ha878542_0         137 KB  conda-forge
    folium-0.12.0              |     pyhd8ed1ab_1          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         227 KB

The following NEW packages will be INSTALLED:

  branca             conda-forge/noarch::branca-0.4.2-pyhd8ed1ab_0
  folium             conda-forge/noarch::folium-0.12.0-pyhd8ed1ab_1

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    

In [2]:
# read the GLA data into a dataframe
df = pd.read_excel('LDD - Housing Completions unit level (final).xlsx', sheet_name="Unit level")

list(df.columns.values)

['Borough',
 'Planning Authority',
 'Borough Reference',
 'Permission type',
 'Existing units',
 'Proposed units',
 'Net unit level',
 'Number of bedrooms',
 'Affordable (Yes/No)',
 'Unit Tenure',
 'Unit provider',
 'Unit Type',
 'Multiple Occupancy (Y/N)',
 'Housing for Older People (Y/N)',
 'Other Sheltered (Y/N)',
 'Plot description',
 'Maximum building height (storeys)',
 'Development type',
 'Permission Status',
 'Permission Date',
 'Completed Date (Res)',
 'Completed Financial Year (Res)',
 'Scheme Name',
 'Site Name/Number',
 'Primary Street Name',
 'Secondary Street(s)',
 'Post Code',
 'Ward',
 'Easting',
 'Northing',
 'Building height (maximum storeys)',
 'Total proposed units',
 'Residential Site Area',
 'Total site area',
 'Development Description']

After reviewing the head and shape of the above table, I want to do the following:
- Use 'Post Code' field for the location reference.
- Use 'Total proposed units' and restrict the values to >6 to eliminate town house conversions. 

In [3]:
# rename column field values so that they are more user friendly
df.rename(columns={'Post Code': 'PostCode', 'Total proposed units': 'TotalUnits', 'Completed Date (Res)': 'Completed'}, inplace=True)

In [4]:
# Reformat date completed to show only the year to make it easier to select developments completed after 2017
df['Completed'] = df['Completed'].dt.year
df.head()

,Borough,Planning Authority,Borough Reference,Permission type,Existing units,Proposed units,Net unit level,Number of bedrooms,Affordable (Yes/No),Unit Tenure,...,Secondary Street(s),PostCode,Ward,Easting,Northing,Building height (maximum storeys),TotalUnits,Residential Site Area,Total site area,Development Description
0,Barking and Dagenham,Barking and Dagenham,0000016,Full,0,17,17,NaN,No,Not Known,...,NaN,RM10 9RD,VILLAGE,549459,184398,NaN,17.0,0.00,0.00,Amended application for demolition of existing...
1,Barking and Dagenham,Barking and Dagenham,0000016,Full,26,0,-26,NaN,No,Not Known,...,NaN,RM10 9RD,VILLAGE,549459,184398,NaN,17.0,0.00,0.00,Amended application for demolition of existing...
2,Barking and Dagenham,Barking and Dagenham,0000041,Full,0,1,1,NaN,No,Not Known,...,NaN,IG11 7DN,GASCOIGNE,544933,183712,NaN,1.0,0.02,0.02,Erection of detached 3 bedroom two storey house
3,Barking and Dagenham,Barking and Dagenham,0000052,Full,0,1,1,3.0,Yes,Intermediate,...,NaN,RM10,VILLAGE,550749,184676,NaN,1.0,0.04,0.04,Erection of two storey 3 bedroom caretakers house
4,Barking and Dagenham,Barking and Dagenham,0000053,Full,0,1,1,1.0,No,Market,...,NaN,IG11 7RW,EASTBURY,545048,183860,NaN,1.0,0.01,0.01,Conversion of ground floor shop to one bedroom...


In [5]:
# Create a new dataframe with only the columns required
Develop = df[['Borough', 'PostCode', 'TotalUnits', 'Completed']]
print(Develop.shape)
Develop.head()

(149856, 4)


,Borough,PostCode,TotalUnits,Completed
0,Barking and Dagenham,RM10 9RD,17.0,2001
1,Barking and Dagenham,RM10 9RD,17.0,2001
2,Barking and Dagenham,IG11 7DN,1.0,2003
3,Barking and Dagenham,RM10,1.0,2001
4,Barking and Dagenham,IG11 7RW,1.0,2000


In [6]:
# Show only developments with >100 & <300 units completed after 2016 (e.g. completed within the last 3 years)
Develop = Develop[(Develop.TotalUnits > 100) & (Develop.Completed > 2016)]
Develop.head(40)

,Borough,PostCode,TotalUnits,Completed
370,Barking and Dagenham,IG11 0XF,10800.0,2017
371,Barking and Dagenham,IG11 0XF,10800.0,2017
372,Barking and Dagenham,IG11 0XF,10800.0,2017
381,Barking and Dagenham,IG11 0XF,10800.0,2017
382,Barking and Dagenham,IG11 0XF,10800.0,2017
383,Barking and Dagenham,IG11 0XF,10800.0,2017
398,Barking and Dagenham,IG11 0XF,10800.0,2017
399,Barking and Dagenham,IG11 0XF,10800.0,2017
400,Barking and Dagenham,IG11 0XF,10800.0,2017
861,Barking and Dagenham,IG11 0JP,290.0,2019


We're seeing a lot of duplicate rows which are likely amendments to the original planning permission.  
We only want one record per development so we will cleanse the data using the drop-duplicates() function.  
We're also seeing a lot of post codes from outer London post codes, so we will restrict the dataframe to inner london postcodes only.

In [7]:
# Eliminates duplicates
Develop.drop_duplicates(keep = False, inplace=True)

# Restrict postcodes to inner London only
inner_london = ["EC", "E", "NE", "N", "NW", "W", "SW", "S", "SE"]
Develop = Develop[Develop['PostCode'].str.startswith(tuple(inner_london))] 
print(Develop.shape)
Develop

(14, 4)


,Borough,PostCode,TotalUnits,Completed
1503,Barnet,NW9 7QP,298.0,2017
1612,Barnet,NW7 1NX,114.0,2019
5682,Barnet,NW9 5HG,714.0,2017
6335,Barnet,NW7 1PZ,133.0,2018
26108,City of London,EC1,231.0,2018
45469,Greenwich,SE10 9HE,645.0,2017
46624,Greenwich,SE18,152.0,2018
46699,Greenwich,SE9 2EY,135.0,2019
56702,Hammersmith and Fulham,W6,744.0,2019
63596,Haringey,N8 0HG,174.0,2019


Another problem identified is the lack of full post codes for some developments.  
This will make mapping the blocks impossible, so we will remove these. 

In [8]:
# remove incomplete postcodes e.g. < 6 characters
Develop = Develop[Develop['PostCode'].str.len() > 6]
Develop

,Borough,PostCode,TotalUnits,Completed
1503,Barnet,NW9 7QP,298.0,2017
1612,Barnet,NW7 1NX,114.0,2019
5682,Barnet,NW9 5HG,714.0,2017
6335,Barnet,NW7 1PZ,133.0,2018
45469,Greenwich,SE10 9HE,645.0,2017
46699,Greenwich,SE9 2EY,135.0,2019
124381,Sutton,SM5 2BT,474.0,2017
130244,Tower Hamlets,E14 6TB,143.0,2019


This is our final clean list of inner city developments.  
We now need to import a dataframe containing latitude and longitude and then merge the two dataframes. 

In [9]:
# Import file containing postcodes
dfGeoloc = pd.read_excel('Geospatial_Coordinates.xlsx')
dfGeoloc

,PostCode,Latitude,Longitude
0,NW7 1NX,51.615455,-0.205672
1,NW7 1PZ,51.613163,-0.212839
2,NW9 7QP,51.576994,-0.240115
3,NW9 5HG,51.595886,-0.252152
4,SE10 9HE,51.499207,-0.011677
5,SE9 2EY,51.442283,0.079589
6,SM5 2BT,51.379153,-0.165565
7,E14 6TB,51.516625,-0.019962


In [10]:
# Merge the two dataframes on the postcode field
df_merge = pd.merge(Develop, dfGeoloc, on='PostCode')
df_merge

,Borough,PostCode,TotalUnits,Completed,Latitude,Longitude
0,Barnet,NW9 7QP,298.0,2017,51.576994,-0.240115
1,Barnet,NW7 1NX,114.0,2019,51.615455,-0.205672
2,Barnet,NW9 5HG,714.0,2017,51.595886,-0.252152
3,Barnet,NW7 1PZ,133.0,2018,51.613163,-0.212839
4,Greenwich,SE10 9HE,645.0,2017,51.499207,-0.011677
5,Greenwich,SE9 2EY,135.0,2019,51.442283,0.079589
6,Sutton,SM5 2BT,474.0,2017,51.379153,-0.165565
7,Tower Hamlets,E14 6TB,143.0,2019,51.516625,-0.019962


In [11]:
address = 'London, UK'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of London are {}, {}.'.format(latitude, longitude))

# create map of London using latitude and longitude values
map_London = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df_merge['Latitude'], df_merge['Longitude'], df_merge['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_London)  
    
map_London

The geograpical coordinates of London are 51.5073219, -0.1276474.
